In [16]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np

movies = {
    "MovieID": [1, 2, 3, 4, 5],
    "Title": ["Inception", "The Dark Knight", "Interstellar", "The Prestige", "Memento"],
    "Genres": ["Sci-Fi, Thriller", "Action, Crime, Drama", "Adventure, Drama, Sci-Fi", "Drama, Mystery, Sci-Fi", "Mystery, Thriller"],
    "Descriptions": [
        "A dream thief who steals corporate secrets through the use of dream-sharing technology is given the inverse task of planting an idea into the mind of a CEO.",
        "When the Batman's mysterious enemy, Joker, leaves a trail of devastating chaos, Batman is forced to face everything he holds near and dear.",
        "A team of explorers travels through a wormhole in search of a new home for humanity.",
        "A prestigious magician suffers personal loss while competing with a rival magician.",
        "A man with short-term memory loss attempts to find his wife's killer."
    ]
}

ratings = {
    "UserID": [1, 1, 1, 2, 2, 3, 3, 4, 4, 5],
    "MovieID": [1, 2, 3, 1, 4, 2, 5, 3, 4, 5],
    "Rating": [5, 4, 4, 5, 4, 5, 3, 4, 5, 4]
}

movies_df = pd.DataFrame(movies)
ratings_df = pd.DataFrame(ratings)

movie_ratings = pd.merge(ratings_df, movies_df, on="MovieID")

user_item_matrix = movie_ratings.pivot_table(index="UserID", columns="Title", values="Rating").fillna(0)

user_similarity = cosine_similarity(user_item_matrix)
user_similarity_df = pd.DataFrame(user_similarity, index=user_item_matrix.index, columns=user_item_matrix.index)

movies_df["Genres"] = movies_df["Genres"].str.replace(" ", "") 
count_vectorizer = CountVectorizer(tokenizer=lambda x: x.split(","))
genre_matrix = count_vectorizer.fit_transform(movies_df["Genres"])
cosine_sim = cosine_similarity(genre_matrix, genre_matrix)

def recommend_collaborative(user_id, user_similarity_df=user_similarity_df, user_item_matrix=user_item_matrix):
    similar_users = user_similarity_df[user_id]
    weighted_ratings = np.dot(similar_users, user_item_matrix.fillna(0))
    avg_ratings = weighted_ratings / similar_users.sum()
    user_movies = user_item_matrix.loc[user_id]
    unrated_movies = user_movies[user_movies == 0]
    recommendations = pd.Series(avg_ratings, index=user_item_matrix.columns).loc[unrated_movies.index]
    recommendations = recommendations.sort_values(ascending=False).head(3)
    return recommendations

def recommend_content(movie_title, cosine_sim=cosine_sim, df=movies_df):
    idx = df[df["Title"] == movie_title].index[0]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:4]  # Skip the first result (itself)
    movie_indices = [i[0] for i in sim_scores]
    return df.iloc[movie_indices][["Title", "Genres", "Descriptions"]]

def interactive_recommendation():
    print("Welcome to the Movie Recommendation System!")
    
    while True:
        print("\nChoose an option:")
        print("1. Content-Based Recommendations (based on a movie title)")
        print("2. Collaborative Filtering Recommendations (based on user ratings)")
        print("0. Exit")
        
        try:
            choice = int(input("Enter your choice (1, 2, or 0 to exit): "))
            if choice == 1:
                print("\nMovies available for recommendation:")
                print(movies_df["Title"].tolist())
                movie_title = input("Enter a movie title from the above list: ").strip()
                if movie_title in movies_df["Title"].values:
                    recommendations = recommend_content(movie_title)
                    print(f"\nContent-Based Recommendations for '{movie_title}':")
                    for _, row in recommendations.iterrows():
                        print(f"- {row['Title']} ({row['Genres']}): {row['Descriptions']}")
                else:
                    print("Invalid movie title. Please try again.")
            
            elif choice == 2:
                print("\nUsers available for recommendations:")
                print(user_item_matrix.index.tolist())
                user_id = int(input("Enter a User ID from the above list: "))
                if user_id in user_item_matrix.index:
                    recommendations = recommend_collaborative(user_id)
                    print(f"\nCollaborative Filtering Recommendations for User {user_id}:")
                    for movie, predicted_rating in recommendations.items():
                        movie_details = movies_df[movies_df["Title"] == movie].iloc[0]
                        print(f"- {movie_details['Title']} ({movie_details['Genres']}): Predicted Rating = {predicted_rating:.2f}")
                else:
                    print("Invalid User ID. Please try again.")
            
            elif choice == 0:
                print("Thank you for using the Movie Recommendation System! Goodbye!")
                break
            
            else:
                print("Invalid choice. Please enter 1, 2, or 0.")

        except ValueError:
            print("Invalid input. Please enter a number (1, 2, or 0).")

interactive_recommendation()


C:\Users\sheet\anaconda\Lib\site-packages\sklearn\feature_extraction\text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Welcome to the Movie Recommendation System!

Choose an option:
1. Content-Based Recommendations (based on a movie title)
2. Collaborative Filtering Recommendations (based on user ratings)
0. Exit


Enter your choice (1, 2, or 0 to exit):  1



Movies available for recommendation:
['Inception', 'The Dark Knight', 'Interstellar', 'The Prestige', 'Memento']


Enter a movie title from the above list:  Inception


...


Users available for recommendations:
[1, 2, 3, 4, 5]


Enter a User ID from the above list:  3



Collaborative Filtering Recommendations for User 3:
- Inception (Sci-Fi,Thriller): Predicted Rating = 1.15
- Interstellar (Adventure,Drama,Sci-Fi): Predicted Rating = 0.92
- The Prestige (Drama,Mystery,Sci-Fi): Predicted Rating = 0.00

Choose an option:
1. Content-Based Recommendations (based on a movie title)
2. Collaborative Filtering Recommendations (based on user ratings)
0. Exit


Enter your choice (1, 2, or 0 to exit):  0


Thank you for using the Movie Recommendation System! Goodbye!
